In [21]:
import os
import glob
import math
import torch
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
from torchnet import meter
from progressbar import * #进度条
from natsort import natsorted
import torch.nn.functional as F #torch是关于运算的包
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集

#如果多gpu运行，屏蔽下一句
os.environ['CUDA_VISIBLE_DEVICES']='2' 

In [35]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features))
        self.add_module('relu1', nn.ReLU(inplace=True))
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size*growth_rate, kernel_size=1, stride=1, bias=False))
        #引入1*1卷积来降低维度 DenseNet-B
        self.add_module('norm2', nn.BatchNorm2d(bn_size*growth_rate))
        self.add_module('relu2', nn.ReLU(inplace=True))
        self.add_module('conv2', nn.Conv2d(bn_size*growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False))
        self.drop_rate = drop_rate
    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)
class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i*growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i+1), layer)
class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features, kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))

In [76]:
class DenseNet(nn.Module):
    def __init__(self, growth_rate=12, block_config=(16, 16, 16),
                 num_init_features=24, bn_size=4, drop_rate=0, num_classes=10):
        super(DenseNet, self).__init__()
        print(num_init_features, block_config, growth_rate)
        #first convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=3, stride=1, padding=1, bias=False))
        ]))
        #Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features, bn_size=bn_size, growth_rate=growth_rate, \
                               drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i+1), block)
            num_features = num_features + num_layers*growth_rate
            if i != len(block_config)-1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features//2)
                self.features.add_module('transition%d' % (i+1), trans)
                num_features = num_features//2
                #DenseNet-C 0.5
        #Final bath norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))
        #Linear layer
        self.classifer = nn.Linear(num_features, num_classes)
        #initialize conv and bn parameters
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0]*m.kernel_size[1]*m.out_channels
                m.weight.data.normal_(0, math.sqrt(2./n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool2d(out, kernel_size=8, stride=1).view(features.size(0), -1)
        out = self.classifer(out)
        return out

In [77]:
def densenet_BC(depth, k, **kwargs):
    N = (depth-4)//6 #total_num_layer = 2 + 2*3*N + 2 =>[in+out+dense_block+trans] 
    model = DenseNet(growth_rate=k, block_config=[N, N, N], num_init_features=2*k, **kwargs)
    return model

In [79]:
net = DenseNet()
#net = densenet_BC(190, 40, num_classes=10)
input = torch.randn(1, 3, 32, 32)
#y = net(input)
#print(net)
#print(y.size())

24 (16, 16, 16) 12


In [49]:
label2int = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
int2lable = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [50]:
class TrainAndValData(Dataset):
    def __init__(self, img_path, csv_path, train=True, transforms=None):
        '''
        获得所有图片路径，并划分训练集、验证集
        '''
        self.train = train
        files = natsorted(glob.glob(img_path + '/*'))
        labels = pd.read_csv(csv_path).values[:, 1]
        files_num = len(files)
        break_point = int(0.9*files_num)
        if self.train:
            self.img_name = files[: break_point]
            self.img_label = labels[: break_point]
        else:
            self.img_name = files[break_point: ]
            self.img_label = labels[break_point: ]
         
   

        #数据增强
        if transforms is None:
            normalize = T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
            #训练集用数据增强
            if self.train:
                self.transforms = T.Compose([
                    #T.RandomCrop(224, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32`，
                    T.Resize(40),
                    T.RandomResizedCrop(32),
                    T.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
                    #T.RandomVerticalFlip(),
                    T.ToTensor(),
                    normalize 
                ])
            else:
                self.transforms = T.Compose([
                    T.Resize(32),
                    #T.CenterCrop(224),#中心裁剪
                    T.ToTensor(),
                    normalize 
                ])
    def __len__(self):
        '''
        返回数据集中所有图片的个数
        '''
        return len(self.img_name)
    def __getitem__(self, index):
        '''
        返回一张图片的数据
        '''
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        label = label2int[self.img_label[index]]
        return img, label
class TestData(Dataset):
    def __init__(self, img_path, transforms=None):
        files = natsorted(glob.glob(img_path + '/*'))
        self.img_name = files
        if transforms is None:
            normalize = T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
            self.transforms =  T.Compose([
                T.Resize(32),
                T.ToTensor(),
                normalize
            ])
    def __len__(self):
        return len(self.img_name)
    def __getitem__(self, index):
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        return img

In [80]:
train_img_path = 'data/train'
csv_path = 'data/trainLabels.csv'
test_img_path = 'data/test'
train_dataset = TrainAndValData(train_img_path, csv_path, train=True)
val_dataset = TrainAndValData(train_img_path, csv_path, train=False)
test_dataset = TestData(test_img_path)

In [81]:
print('train len is %d' % len(train_dataset))
print(train_dataset[0][0].shape)
print(train_dataset[0][1])#打印标签
print('val len is %d' % len(val_dataset))
print(val_dataset[0][0].shape)
print(val_dataset[0][1])
print('test len is %d' % len(test_dataset))
print(test_dataset[0].shape)

train len is 45000
torch.Size([3, 32, 32])
6
val len is 5000
torch.Size([3, 32, 32])
7
test len is 300000
torch.Size([3, 32, 32])


In [84]:
# 超参数设置
EPOCH = 135   #遍历数据集次数
BATCH_SIZE = 120      #批处理尺寸(batch_size)
#LR = 0.001        #学习率
lr = 0.001
lr_decay = 0.995
weight_decay = 1e-4
model_path = 'model/densenet/wide_resnet.pkl'

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8) 

#device = torch.device("cuda:2")
#net = ResNet_18_34(34)

net = DenseNet()
#net = net.to(device)
##单GPU
#net = net.cuda()
##多GPU
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net, device_ids=[0,1])
net = net.cuda()
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
#optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
#保存模型判断条件
max_val_acc = 0
pre_epoch = 0
max_interval_epoch = 20
pre_train_loss = 100000

print("Start Training...")
for epoch in range(200):
    #训练集
    train_loss = 0
    train_count = 0
    net.train()
    for i, data in enumerate(trainloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #更新指标
        train_count += labels.size(0)
        train_loss += loss.item()
    train_loss /= train_count

    #验证集
    val_acc = 0
    val_loss = 0
    val_count = 0
    net.eval()
    for i, data in enumerate(valloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels =  inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        #更新指标
        val_count += 1
        val_loss += loss.item()
        _, predict = outputs.max(1)
        val_count += labels.size(0)
        val_acc += (predict == labels).sum().item()
    val_acc /= val_count
    val_loss /= val_count
    # print the loss and accuracy
    print('the epoch %d, the train loss is %f, the test loss is %f, the test acc is %f' % (epoch, train_loss, val_loss, val_acc))

    #保存模型
    if val_acc > max_val_acc:
        max_val_acc = val_acc
        pre_epoch = epoch
        torch.save(net, model_path)#保存整个神经网络的的结构信息和模型参数信息，save的对象是网络net
    if epoch - pre_epoch > max_interval_epoch:
        print('early stop')
        break

    #如果损失不载下降，则降低学习率
    if train_loss > pre_train_loss:
        lr = lr*lr_decay
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    pre_train_loss = pre_train_loss
print("Done Training!")

24 (16, 16, 16) 12
Start Training...
the epoch 0, the train loss is 0.017720, the test loss is 0.016517, the test acc is 0.285006


/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type DenseNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type _DenseBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type _DenseLayer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type _Transition. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". I

the epoch 1, the train loss is 0.016157, the test loss is 0.015764, the test acc is 0.290559
the epoch 2, the train loss is 0.015647, the test loss is 0.015213, the test acc is 0.308409
the epoch 3, the train loss is 0.015378, the test loss is 0.014789, the test acc is 0.339151
the epoch 4, the train loss is 0.015141, the test loss is 0.014662, the test acc is 0.337763
the epoch 5, the train loss is 0.014944, the test loss is 0.014586, the test acc is 0.346886
the epoch 6, the train loss is 0.014778, the test loss is 0.014408, the test acc is 0.348473
the epoch 7, the train loss is 0.014589, the test loss is 0.014034, the test acc is 0.365133
the epoch 8, the train loss is 0.014448, the test loss is 0.013936, the test acc is 0.370885
the epoch 9, the train loss is 0.014309, the test loss is 0.013692, the test acc is 0.383181
